In [7]:
import glob
import os
import mne
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy.ma as ma
data_dir = 'data_meg'
subj = "R2210"
dtype = "raw"
raw = mne.io.read_raw_fif('data_meg/R2210_raw_ica.fif', preload=True)
save_dir = 'data_meg'

Opening raw data file data_meg/R2210_raw_ica.fif...


/var/folders/4s/v49tr3_d3vn5jcm7fg7z3cv00000gp/T/ipykernel_56773/3582625413.py:10: RuntimeWarning: This filename (data_meg/R2210_raw_ica.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif('data_meg/R2210_raw_ica.fif', preload=True)


    Range : 0 ... 2779999 =      0.000 ...  2779.999 secs
Ready.
Opening raw data file /Users/labadmin/meg_decoding/data_meg/R2210_raw_ica-1.fif...
    Range : 2780000 ... 2954999 =   2780.000 ...  2954.999 secs
Ready.
Reading 0 ... 2954999  =      0.000 ...  2954.999 secs...


In [8]:
import json
data_log_dir = 'data_log/R2210'
json_files = glob.glob(os.path.join(data_log_dir, '*.json'))

# Initialize an empty list to store DataFrames
dataframes = []

# Iterate over each JSON file and read it into a DataFrame
for json_file in json_files:
    with open(json_file, 'r') as file:
        data = json.load(file)
        trial_data = data['trial_data']
    df = pd.json_normalize(trial_data)
    dataframes.append(df)
if dataframes:
    combined_data = pd.concat(dataframes, ignore_index=True)
    print(combined_data)
else:
    print("No valid JSON files found.")

                                                events  \
0    [{'time': 269.2678911240073, 'event': 'start',...   
1    [{'time': 299.0857983099995, 'event': 'start',...   
2    [{'time': 338.2656750899041, 'event': 'start',...   
3    [{'time': 358.6425380120054, 'event': 'start',...   
4    [{'time': 415.35983730095904, 'event': 'start'...   
..                                                 ...   
115  [{'time': 2523.152022905997, 'event': 'start',...   
116  [{'time': 2543.537114879, 'event': 'start', 'c...   
117  [{'time': 2563.9295538810547, 'event': 'start'...   
118  [{'time': 2594.5419432560448, 'event': 'start'...   
119  [{'time': 2607.757046270999, 'event': 'start',...   

                                          white_tiles  \
0            [[5, 3], [4, 3], [4, 1], [3, 2], [6, 2]]   
1    [[1, 3], [3, 3], [3, 2], [1, 6], [1, 5], [1, 4]]   
2                    [[4, 1], [4, 0], [4, 5], [3, 3]]   
3                            [[4, 6], [3, 6], [4, 3]]   
4                 

In [9]:
raw.filter(1, 40, method='iir')

Filtering raw data in 3 contiguous segments
Setting up band-pass filter from 1 - 40 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 1.00, 40.00 Hz: -6.02, -6.02 dB



<Raw | R2210_raw_ica.fif, 193 x 2955000 (2955.0 s), ~4.25 GB, data loaded>

In [10]:
events = mne.find_events(raw, stim_channel='STI 014', output='onset', shortest_event=1)
event_id = {
    'start': 1,
    'move': 2,
    'reveal_red': 4,
    'reveal_white': 8,
    'done': 16,
}
epochs = mne.Epochs(raw, events, event_id, tmin=-0.2, tmax=1.0, baseline=(None, 0), preload=True)

print(epochs)

5438 events found on stim channel STI 014
Event IDs: [ 1  2  4  8 16]
Not setting metadata
5438 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 5438 events and 1201 original time points ...
0 bad epochs dropped
<Epochs | 5438 events (all good), -0.2 – 1 s (baseline -0.2 – 0 s), ~9.39 GB, data loaded,
 'start': 120
 'move': 4191
 'reveal_red': 617
 'reveal_white': 390
 'done': 120>


In [14]:
start_events = events[events[:, 2] == event_id['start']]
done_events = events[events[:, 2] == event_id['done']]
sfreq = raw.info['sfreq']  # Sampling frequency

# Initialize a list to store trial information
trial_info = []

done_idx = 0

for start_event in start_events:
    start_sample = start_event[0]
    
    # Find the corresponding 'done' event
    while done_idx < len(done_events) and done_events[done_idx, 0] <= start_sample:
        done_idx += 1
    if done_idx < len(done_events):
        done_sample = done_events[done_idx, 0]
        done_idx += 1

        # Calculate tmin and tmax for the epoch
        tmin = -0.2  # 0.2 s before 'start'
        tmax = (done_sample - start_sample) / sfreq + 1.0  # Duration from 'start' to 1 s after 'done'

        # Store trial information
        trial_info.append({
            'event_sample': start_sample,
            'tmin': tmin,
            'tmax': tmax
        })
max_tmax = max([info['tmax'] for info in trial_info])
print(f"Maximum epoch duration: {max_tmax:.2f} seconds")


Maximum epoch duration: 78.10 seconds


In [15]:
new_events = np.array([[info['event_sample'], 0, event_id['start']] for info in trial_info])
tmin = -0.2  
# Create the Epochs object
epochs = mne.Epochs(raw, new_events, event_id={'start': event_id['start']},
                    tmin=tmin, tmax=max_tmax, baseline=(None, 0), preload=True)


Not setting metadata
120 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 120 events and 78303 original time points ...
10 bad epochs dropped


In [ ]:
# After creating the epochs
epochs = mne.Epochs(raw, new_events, event_id={'start': event_id['start']},
                    tmin=tmin, tmax=max_tmax, baseline=(None, 0), preload=True, reject_by_annotation=False)

# Check the number of trials and epochs
print(f"Number of trials in trial_info: {len(trial_info)}")
print(f"Number of epochs: {len(epochs)}")

# Check for dropped epochs
if len(epochs.drop_log) > 0:
    print("Some epochs were dropped:")
    for idx, reason in enumerate(epochs.drop_log):
        if reason != ():
            print(f"Epoch {idx} was dropped due to: {reason}")

# Get the indices of epochs that were kept
selected_indices = epochs.selection

# Adjust trial_info to include only the selected trials
selected_trial_info = [trial_info[i] for i in selected_indices]

# Get the epochs data
data = epochs.get_data()  # Shape: (n_epochs, n_channels, n_times)

# Initialize a mask for valid data points
valid_data_mask = np.zeros(data.shape, dtype=bool)

# Loop over the selected trials
for idx, info in enumerate(selected_trial_info):
    # Calculate the number of samples for this trial
    duration_samples = int((info['tmax'] - tmin) * sfreq)
    valid_data_mask[idx, :, :duration_samples] = True

# Create a masked array
masked_data = ma.masked_array(data, mask=~valid_data_mask)


In [16]:
import numpy.ma as ma

# Get the epochs data
data = epochs.get_data()  # Shape: (n_epochs, n_channels, n_times)
times = epochs.times

# Initialize a mask for valid data points
valid_data_mask = np.zeros(data.shape, dtype=bool)

for idx, info in enumerate(trial_info):
    # Calculate the number of samples for this trial
    duration_samples = int((info['tmax'] - tmin) * sfreq)
    valid_data_mask[idx, :, :duration_samples] = True

# Create a masked array
masked_data = ma.masked_array(data, mask=~valid_data_mask)


IndexError: index 110 is out of bounds for axis 0 with size 110

In [25]:
epochs.plot(n_epochs=5, n_channels=30,events=True)


Not setting metadata
120 matching events found
No baseline correction applied
0 projection items activated


In [6]:
meg_data = epochs.get_data(picks=["mag", "grad"])